In [ ]:
# %% [markdown]
# # Animation of bioRSP Algorithm with CDF Differences Up to Current Angle

# %%
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Import functions from the bioRSP module
from ipynb.fs.full.biorsp import (
    find_foreground_background_points,
    convert_to_polar,
    in_scanning_range,
    compute_cdfs,
    compute_area,
    compute_cdf,  # Ensure compute_cdf is imported
)

# %%
os.makedirs("animations", exist_ok=True)

# %%
# Load your data
dge_matrix = pd.read_csv("data/MCA2_filtered.dge.txt", sep="\t", index_col=0)
tsne_results = pd.read_csv("embeddings/tsne_results.csv").to_numpy()
dbscan_results = pd.read_csv("embeddings/tsne_dbscan_results.csv")

# %%
# Define parameters
gene_name = "Tnnt2"
threshold = 10
clusters = [1]
scanning_window = np.pi
num_scanning_angles = 360  # Number of frames in animation
num_bins_within_window = 1000  # Number of bins within scanning window
angle_range = np.array([0.0, 2.0 * np.pi])
mode = "absolute"

# %%
# Find foreground and background points
foreground_points, background_points = find_foreground_background_points(
    gene_name=gene_name,
    dge_matrix=dge_matrix,
    tsne_results=tsne_results,
    threshold=threshold,
    dbscan_df=dbscan_results,
    selected_clusters=clusters,
)

# %%
# Set the vantage point
vantage_point = background_points.mean(axis=0)

# %%
# Convert points to polar coordinates
fg_r, fg_theta = convert_to_polar(foreground_points, vantage_point)
bg_r, bg_theta = convert_to_polar(background_points, vantage_point)

# Ensure angles are within [0, 2π]
fg_theta = fg_theta % (2 * np.pi)
bg_theta = bg_theta % (2 * np.pi)

# %%
# Calculate scanning angles
theta_min = angle_range[0]
theta_max = angle_range[1]
theta_k_list = (
    theta_min
    + np.arange(num_scanning_angles) * (theta_max - theta_min) / num_scanning_angles
)

# Determine the maximum radius for the scanning window
radius_max = max(np.max(fg_r), np.max(bg_r))

all_diffs = []

# %%
# Set up the figure and axes for animation with three panels
fig = plt.figure(figsize=(15, 5))  # Increased width to accommodate third panel
gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 1], wspace=0.5)
ax1 = fig.add_subplot(gs[0], projection="polar")
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2], projection="polar")  # Third panel for cumulative CDF differences

# Function to plot the scanning window on the polar plot
def plot_scanning_window(ax, start_angle, end_angle, radius_max):
    # Handle wrap-around
    if start_angle < end_angle:
        theta_window = np.linspace(start_angle, end_angle, 100)
    else:
        # Wrap-around case
        theta_window = np.linspace(start_angle, end_angle + 2 * np.pi, 100)
        theta_window = theta_window % (2 * np.pi)
    # Create the corresponding r values
    r_window = np.full_like(theta_window, radius_max)
    # Add the origin to close the polygon
    theta_polygon = np.concatenate(
        ([start_angle], theta_window, [end_angle], [start_angle])
    )
    r_polygon = np.concatenate(([0], r_window, [0], [0]))
    # Plot the scanning window
    ax.fill(theta_polygon, r_polygon, color="yellow", alpha=0.3)

def animate(i):
    try:
        angle = theta_k_list[i]
        theta_shift = angle - scanning_window / 2
        start_angle = theta_shift % (2 * np.pi)
        end_angle = (theta_shift + scanning_window) % (2 * np.pi)

        ax1.clear()
        ax2.clear()
        ax3.clear()  # Clear the third axis

        # Radar scanning demo (Left Side) - Polar Plot
        # Plot background points
        ax1.scatter(bg_theta, bg_r, color="grey", s=1, label="Background")
        # Plot foreground points
        ax1.scatter(fg_theta, fg_r, color="red", s=1, label="Foreground")

        # Highlight scanning window
        plot_scanning_window(ax1, start_angle, end_angle, radius_max)

        ax1.set_ylim(0, radius_max)
        ax1.set_title(f"Scanning Angle: {np.degrees(angle)%360:.2f}°")
        # Show gridlines and spines
        ax1.grid(True)
        # Set tick marks
        ax1.set_xticks(np.linspace(0, 2 * np.pi, 8, endpoint=False))  # Every 45 degrees
        ax1.set_yticks(np.linspace(0, radius_max, 5))
        ax1.legend(loc="upper right", bbox_to_anchor=(1.1, 1.1))

        # Compute CDFs for the current scanning window using provided functions
        fg_in_window = in_scanning_range(fg_theta, angle, scanning_window)
        bg_in_window = in_scanning_range(bg_theta, angle, scanning_window)

        fg_projection = fg_theta[fg_in_window]
        bg_projection = bg_theta[bg_in_window]

        # Use provided functions to compute histograms and CDFs
        fg_cdf, bg_cdf = compute_cdfs(
            fg_projection,
            bg_projection,
            angle,
            scanning_window,
            num_bins_within_window,
            mode,
        )
        bin_edges = np.linspace(0, 2 * np.pi, num_bins_within_window + 1)

        # Prepare bin edges and centers
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        # CDFs plot (Middle Panel)
        if np.any(bg_cdf):
            # Plot background CDF
            ax2.step(
                bin_centers,
                bg_cdf,
                where="mid",
                color="grey",
                label="Background CDF",
            )

        if np.any(fg_cdf):
            # Plot foreground CDF
            ax2.step(
                bin_centers,
                fg_cdf,
                where="mid",
                color="red",
                label="Foreground CDF",
            )

        # Handle case when there is no foreground data
        if not np.any(fg_cdf):
            ax2.text(
                0.5,
                0.5,
                "No Foreground Data in this window",
                horizontalalignment="center",
                verticalalignment="center",
                transform=ax2.transAxes,
            )

        ax2.legend()
        ax2.set_xlabel("Theta (radians)")
        ax2.set_ylabel("CDF")
        ax2.set_title("CDFs within Scanning Window")
        ax2.set_xlim(bin_centers[0], bin_centers[-1])
        ax2.set_ylim(0, 1)
        ax2.grid(True)
        ax2.set_aspect(aspect="auto", adjustable="box")

        angle_mod = angle % (2 * np.pi)
        if angle_mod == 0:
            angle_mod = 2 * np.pi

        area = compute_area(fg_cdf, bg_cdf, scanning_window)
        all_diffs.append(area)

        ax3.plot(theta_k_list[:i+1], all_diffs[:i+1], color='blue')
        ax3.legend()
        ax3.set_xlabel("Theta (radians)")
        ax3.set_ylabel("Difference")
        ax3.set_title("CDF Differences Up to Current Angle")
        ax3.set_xlim(0, angle_mod)
        ax3.set_ylim(0, 1)
        ax3.grid(True)
        ax3.set_aspect(aspect="auto", adjustable="box")

    except Exception as e:
        print(f"Exception at frame {i}: {e}")
        raise e  # Re-raise the exception to stop the animation

# %%
# Create the animation
anim = FuncAnimation(fig, animate, frames=num_scanning_angles, interval=100)

# Save the animation as a GIF
# anim.save(f"animations/{gene_name}_rsp_animation.gif", writer="pillow", fps=10)

# To display the animation in the notebook (uncomment if needed)
from IPython.display import HTML
HTML(anim.to_jshtml())

/var/folders/g4/m4yltzqj0c79mflv15hj0b3w0000gn/T/ipykernel_71713/3910840254.py:206: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax3.legend()
Animation size has reached 20986522 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.
